In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

train = pd.read_csv("train.csv")
target = train['label']
test = pd.read_csv("test.csv")

global image 
global target
global test_images

image = train.iloc[:, 1:].values
image=np.multiply(image, 1.0/255.0)

test_images = test.iloc[:, :].values
test_images = np.multiply(test_images, 1.0/255.0)
# Any results you write to the current directory are saved as output.

In [18]:
def dense_to_one_hot(label):
    y = np.zeros((label.shape[0], 10))
    y[np.arange(label.shape[0]), label] = 1
    return y

In [19]:
target = dense_to_one_hot(target)

In [20]:
learning_rate = 0.0001
training_iters = 25000
batch_size = 100
display_step = 1
dropout = 0.75
n_classes = 10
n_input = 784
img_width = 28
img_height = 28
epochs = 0


In [21]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

In [22]:
def conv2d(x, W, b):
    x = tf.nn.conv2d(x, W, strides = [1, 1, 1, 1], padding = 'SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [23]:
def max_pool(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')

In [24]:
def conv_net(x, weights, biases, dropout):
    x = tf.reshape(x, shape = [-1, 28, 28, 1])
    
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = max_pool(conv1)
    
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = max_pool(conv2)
    
    fc = tf.reshape(conv2, [-1, weights['wd'].get_shape().as_list()[0]])
    fc = tf.add(tf.matmul(fc, weights['wd']), biases['bd'])
    fc = tf.nn.relu(fc)
    
    out = tf.add(tf.matmul(fc, weights['out']), biases['out'])
    return out

In [25]:
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd' : tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, 10]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd' : tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([10]))
}

In [42]:
pred = conv_net(x, weights, biases, keep_prob)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

final = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(final, tf.float32))

predict = tf.argmax(pred, 1)

In [27]:
def next_batch():
    global epochs
    global start
    global image
    global target
    if (start+1)*batch_size > image.shape[0]:
        epochs+=1
        temp = np.arange(image.shape[0])
        np.random.shuffle(temp)
        image = image[temp]
        target = target[temp]
        start = 0
    return image[start*batch_size:(start+1)*batch_size], target[start*batch_size:(start+1)*batch_size]

In [28]:
init = tf.initialize_all_variables()
sess = tf.InteractiveSession()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [29]:
global start
start = 0
for i in range(training_iters):
    image_next, target_next = next_batch()
    start+=1
    sess.run(optimizer, feed_dict={x:image_next, y: target_next, keep_prob: dropout})
    acc = sess.run(accuracy, feed_dict={x:image_next, y: target_next, keep_prob: 1.0})
    if (i+1)%display_step==0:
        print("Accuracy: {0} Step {1} start {2}".format(acc, i+1, start))
    if i+1==display_step*10:
        display_step*=10
print("Optimization finished! Epochs: {0}".format(epochs))

Accuracy: 0.05999999865889549 Step 1 start 1
Accuracy: 0.10999999940395355 Step 2 start 2
Accuracy: 0.10000000149011612 Step 3 start 3
Accuracy: 0.05999999865889549 Step 4 start 4
Accuracy: 0.05999999865889549 Step 5 start 5
Accuracy: 0.07999999821186066 Step 6 start 6
Accuracy: 0.07000000029802322 Step 7 start 7
Accuracy: 0.10999999940395355 Step 8 start 8
Accuracy: 0.07999999821186066 Step 9 start 9
Accuracy: 0.10000000149011612 Step 10 start 10
Accuracy: 0.10999999940395355 Step 20 start 20
Accuracy: 0.07999999821186066 Step 30 start 30
Accuracy: 0.10999999940395355 Step 40 start 40
Accuracy: 0.14000000059604645 Step 50 start 50
Accuracy: 0.12999999523162842 Step 60 start 60
Accuracy: 0.2199999988079071 Step 70 start 70
Accuracy: 0.18000000715255737 Step 80 start 80
Accuracy: 0.15000000596046448 Step 90 start 90
Accuracy: 0.23000000417232513 Step 100 start 100
Accuracy: 0.5299999713897705 Step 200 start 200
Accuracy: 0.550000011920929 Step 300 start 300
Accuracy: 0.7200000286102295 

In [43]:
predicted = np.zeros(test_images.shape[0])
global test_images
for i in range(test_images.shape[0]//batch_size):
    predicted[i*batch_size: (i+1)*batch_size] = predict.eval(feed_dict={x: test_images[i*batch_size : (i+1)*batch_size], \
                                                                                keep_prob: 1.0})

In [45]:
np.savetxt('submission_softmax.csv', 
           np.c_[range(1,len(test_images)+1),predicted], 
           delimiter=',', 
           header = 'ImageId,Label', 
           comments = '', 
           fmt='%d')